In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import os

from tqdm import tqdm

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import colors, cm, pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
label = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
file_path_train = []
file_path_test = []
for dirname, _, filenames in os.walk('/kaggle/input/seti-breakthrough-listen/test'):
    for filename in filenames:
        file_path_test.append(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk('/kaggle/input/seti-breakthrough-listen/train'):
    for filename in filenames:
        file_path_train.append(os.path.join(dirname, filename))  

In [ ]:
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
df['path'] = df['id'].apply(lambda x: 
                            str('../input/seti-breakthrough-listen' 
                                + '/train/' 
                                + str(x[0]) 
                                + '/' + str(x)) 
                            + '.npy')

df_test = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
df_test['path'] = df_test['id'].apply(lambda x: 
                            str('../input/seti-breakthrough-listen' 
                                + '/test/' 
                                + str(x[0]) 
                                + '/' + str(x)) 
                            + '.npy')

In [ ]:
sig = df[df.target == 1].sample(df[df.target == 1].shape[0] - 22)
no_sig = df[df.target == 0].sample(df[df.target == 1].shape[0] - 22 + 64*4)
new_df = pd.concat([sig, no_sig], axis=0)

In [ ]:
data = np.load(df['path'][0])
smth = (data[3][:256] - data[3][:256].min()) / (data[3][:256].max() - data[3][:256].min())

In [ ]:

img= (data[1][:256]) 
plt.imshow(img.astype("float"), cmap=plt.cm.binary, vmin=-0.5, vmax=10)
plt.show()

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_path, transform=None, target_transform=None):
        self.img_labels = annotations_file
        self.img_path = img_path
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image = np.load(self.img_path[idx])
        label = self.img_labels[idx]
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        
        x1 = image[0][:256]
        x2 = image[1][:256]
        x3 = image[2][:256]
        x4 = image[3][:256]
        x5 = image[4][:256]
        x6 = image[5][:256] 
        
        image = [(x1-x1.min())/(x1.max()-x1.min()),
                 (x2-x2.min())/(x2.max()-x2.min()),
                 (x3-x3.min())/(x3.max()-x3.min()),
                 (x4-x4.min())/(x4.max()-x4.min()),
                 (x5-x5.min())/(x5.max()-x5.min()),
                 (x6-x6.min())/(x6.max()-x6.min())]
        image = torch.tensor(image)
        label = torch.tensor(label)
            
        return image, label

    
batch_size = 64    
    
features_train, features_test, targets_train, targets_test = train_test_split(new_df['path'].values, 
                                                                              new_df['target'].values, 
                                                                              test_size=0.2)   
    
train_loader = CustomImageDataset(targets_train ,features_train)
test_loader = CustomImageDataset(targets_test, features_test)

train_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True)

for X, y in test_loader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

In [ ]:
# batch_size = 100    
    
# features_train, features_test, targets_train, targets_test = train_test_split(df.head(1000).path.values, 
#                                                                               df.head(1000).target.values, 
#                                                                               test_size=0.2)   
    
# train_loader = CustomImageDataset(targets_train ,features_train)
# test_loader = CustomImageDataset(targets_test, features_test)

# train_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True)

In [ ]:
#for i in range(4, 7):
#    x1, x2, x3, x4, x5, x6, label = CustomImageDataset.__getitem__(Dataset, i)
#    print(x1, x2, x3, x4, x5, x6, label)
#CustomImageDataset(df['target'].values, df['path'].values).__len__()
#train_loader

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.feature1 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=48, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(4),
            nn.Conv2d(in_channels=48, out_channels=96, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(4),
            nn.Conv2d(in_channels=96, out_channels=192, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(4)
        )
        
        self.feature2 = nn.Sequential(
            nn.Linear(192 * 4 * 4, 3100),
            nn.ReLU(),
            nn.Linear(3100, 750),
            nn.ReLU(),
            nn.Linear(750, 200),
            nn.ReLU(),
            nn.Linear(200, 50),
            nn.ReLU(),
            nn.Linear(50, 1),
            nn.Sigmoid()
        )
        

        
    def forward(self, image):
        
        x = self.feature1(image.float())
        
        x = self.feature2(x.view(-1, 192 * 4 * 4))
        
        
        return x

model = CNNModel()
model.to(device)
print(model)

In [ ]:
#for a,b in train_loader:
#    print(a.shape)
#    a.to(device)
#    break
#print(a[1,:,:,:].view(1,6,256,256).shape)
#data_r = a[1,:,:,:].view(1,6,256,256)
#data_r = data_r.to(device)
#pred = model(data_r)

In [ ]:
#pred

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in tqdm(enumerate(dataloader), position=0, leave=True, desc='Evaluating'):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        loss = loss_fn(pred.view(batch_size), y.float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in tqdm(dataloader, position=0, leave=True, desc='Evaluating'):
            X, y = X.to(device), y.to(device)
            pred = model(X.float())

            test_loss += loss_fn(pred.view(batch_size), y.float()).item()
            
            correct += (pred.argmax(1) == y.float()).type(torch.float).sum().item()
                      
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 10
for t in range(epochs):
    #print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model)
print("Done!")

In [ ]:
torch.save(model.state_dict(),f'my_model_{epochs}.pt')

In [ ]:
batch_size = 64
test_load = CustomImageDataset(df_test['target'], df_test['path'])
test_load = DataLoader(test_load, batch_size=batch_size, shuffle=False)

In [ ]:
model.eval()
    
final_targets = []
final_outputs = []
    
with torch.no_grad():
        
    for data in tqdm(test_load, position=0, leave=True, desc='Evaluating'):
        inputs = data[0]
        targets = data[1]
        inputs = inputs.to(device, dtype=torch.float)
            
        output = model(inputs)
            
        output = output.detach().cpu().numpy().tolist()
            
        final_outputs.extend(output)

In [ ]:
submission = pd.DataFrame(final_outputs, columns = ['target'])
submission = pd.concat([df_test['id'], submission], axis=1) 

In [ ]:
submission.to_csv('submission.csv', index=False)